# Basic TensorFlow
This is a note focus on understanding what is a Tensor and how to use it in TensorFlow. Most of notes are excerpted from articles (source links are provided). 

From [Understanding Tensorflow using Go – P. Galeone’s blog](https://pgaleone.eu/tensorflow/go/2017/05/29/understanding-tensorflow-using-go/)

**Understand Tensorflow structure**

Let’s repeat what Tensorflow is (kept from the [Tensorflow website](https://www.tensorflow.org/) , the emphasis is mine):

> *TensorFlow™ is an open source software library for numerical computation using data flow graphs. Nodes in the graph **represent** mathematical operations, while the graph edges **represent** the multidimensional data arrays (tensors) communicated between them.*

We can think of Tensorflow as a descriptive language, a bit like SQL, in which you describe what you want and let the underlying engine (the database) parse your query, check for syntactic and semantic errors, convert it to its private representation, optimize it and compute the results: all this to give you the correct results.

Therefore, what we really do when we use any of the available APIs is to describe a graph: the evaluation of the graph starts when we place it into a Session and explicitly decide to Run the graph within the Session.

So, to better understand TensorFlow, we need to understand its: 

- edge node, a Tensor, which holds values,
- graph, computational network, which describes the algorithm.

## Background - Numpy Data Type

In [0]:
import numpy as np

In [31]:
# a scalar

# create an object name=b and value=3, 
a = np.int(3)
print('int', a)
# print(a.size)
# print(a.shape), this will give error, scalar has no size and shape

a = np.int(5)   # can re-assign
print('int', a)

b = np.float32(3.0)
print('float32', b)
# print(b.shape), object can be a float, but it is still a scalar, which has no size and shape
print(b.dtype, b)   # use dtype for data type 

c = np.str('abcd')
print(c)
# print(d.dtype) -> AttributeError: 'str' object has no attribute 'dtype'

int 3
int 5
float32 3.0
float32 3.0
abcd


In [32]:
# a vector

d = np.arange(12)
print(d)
print('type is', d.dtype)
print('shape is', d.shape)
# without print, it will only display the last value

[ 0  1  2  3  4  5  6  7  8  9 10 11]
type is int64
shape is (12,)


a'D0 is 12; rank is 1 and there is not D1. This is different from (12, None); the rank is 2 and D1 is unknown. 

In [33]:
# scalar
e1 = np.int64(1)
print(e1.dtype, e1)
print(e1.shape)

e2 = np.int(1)
print(e2)

# vector
e0 = np.array([1])
print(e0.dtype, e0)
print(e0.shape)

data3 = [1, 2, 3]   # list allows only sequential access
print(type(data3))  # find out type
e3 = np.array(data3)
print(e3[0])

f = np.array([(1,2,3), (4,5,6)], dtype = float)
print('vector', f)
print('vector', f[0])

print('ndim', f.ndim)
print('shape-0', f.shape[0])
print('shape-1', f.shape[1])

# matrix
g = np.array([[(1,2,3), (4,5,6)], [(7,8,9), (10,11,12)]], dtype = float)
print('matrix', g)
print('matrix dtype', g.dtype)
print('matrix - 2nd array', g[1])

print('shape', g.shape)  # ===> (2, 2, 3) 2 by 2 matrix with 3 elements in each array, total 2x2x3 = 12 data

int64 1
()
1
int64 [1]
(1,)
<class 'list'>
1
vector [[1. 2. 3.]
 [4. 5. 6.]]
vector [1. 2. 3.]
ndim 2
shape-0 2
shape-1 3
matrix [[[ 1.  2.  3.]
  [ 4.  5.  6.]]

 [[ 7.  8.  9.]
  [10. 11. 12.]]]
matrix dtype float64
matrix - 2nd array [[ 7.  8.  9.]
 [10. 11. 12.]]
shape (2, 2, 3)


A data buffer defined/indexed by original creation, which will be re-indexed. See   
[what does -1 means in numpy reshape](https://stackoverflow.com/questions/18691084/what-does-1-mean-in-numpy-reshape)  
[shape and reshape here](https://stackoverflow.com/questions/22053050/difference-between-numpy-array-shape-r-1-and-r) 

we can have 
- a shape of () and be sure to work with a scalar, 
- a shape of (10) and be sure to work with a vector of size 10, 
- a shape of (10,2) and be sure to work with a matrix with 10 rows and 2 columns. 

In [34]:
# reshape and slice
i = np.arange(0,12)
print(i)
print('shape', i.shape)

k = print('row \n', i.reshape(1,-1))      # column unknown
l = print('column \n', i.reshape(-1,1))   # row unknown

j = i.reshape(3,4)
print(j)
print('shape', j.shape)
print(i.reshape(3,-1))   # 3 rows, -1 for unknown number of columns, same as above (3,4)

print('default back', j.reshape(-1))

print(i.reshape(2,-1))   # -1 for unknown
print('option-C \n', np.reshape(j, (2, 6), 'C'))           # C-like index ordering, row first, default
print('option-F \n', np.reshape(j, (2, -1), order='F'))    # Fortran-like index ordering, column first

[ 0  1  2  3  4  5  6  7  8  9 10 11]
shape (12,)
row 
 [[ 0  1  2  3  4  5  6  7  8  9 10 11]]
column 
 [[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
shape (3, 4)
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
default back [ 0  1  2  3  4  5  6  7  8  9 10 11]
[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]]
option-C 
 [[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]]
option-F 
 [[ 0  8  5  2 10  7]
 [ 4  1  9  6  3 11]]


## What is a Tensor?

From:  
[Tensors  |  TensorFlow  |  TensorFlow](https://www.tensorflow.org/guide/tensors) document.

[Understanding Tensorflow’s tensors shape: static and dynamic – P. Galeone’s blog](https://pgaleone.eu/tensorflow/2018/07/28/understanding-tensorflow-tensors-shape-static-dynamic/)  
[TensorFlow: Shapes and dynamic dimensions – metaflow-ai](https://blog.metaflow.fr/shapes-and-dynamic-dimensions-in-tensorflow-7b1fe79be363)

Very briefly, a tensor is an N-dimensional array containing the same type of data (int32, bool, etc.): All you need to describe a tensor fully is its data type and the value of each of the N dimension. 

Every tensor has a name, a type, a rank and a shape.

* The **name** uniquely identifies the tensor in the computational graphs (for a complete understanding of the importance of the tensor name and how the full name of a tensor is defined, I suggest the reading of the article [Understanding Tensorflow using Go](https://pgaleone.eu/tensorflow/go/2017/05/29/understanding-tensorflow-using-go/) ).
* The **type** is the data type of the tensor, e.g.: a tf.float32, a tf.int64, a tf.string, …
* The **rank**, in the Tensorflow world (that’s different from the mathematics world), is just the number of dimension of a tensor, e.g.: a scalar has rank 0, a vector has rank 1, …
* The **shape** is the number of elements in each dimension, e.g.: a scalar has a rank 0 and an empty shape(), a vector has rank 1 and a shape of(D0), a matrix has rank 2 and a shape of(D0, D1)and so on.

That’s why we describe a tensor with what we call a shape: it is a list, tuple or TensorShape of numbers containing the size of each dimension of our tensor, for example:

> For a tensor of n dimensions: (**D**0, **D**1, …, **D**n-1)  
> For a tensor of size **W** x **H** (usually called a matrix): (**W**, **H**)  
> For a tensor of size **W** (usually called a vector): (**W**,)  
> For a simple scalar (those are equivalent): () or (1,)

> Note: (**D***, **W** and **H** are integers)

> Note on the vector (1-D tensor): it is impossible to determine if a vector is a row or column vector by looking at the vector shape in TensorFlow, and in fact, it doesn’t matter. For more information please look at this stack overflow answer about NumPy notation ( which is roughly the same as TensorFlow notation): http://stackoverflow.com/questions/22053050/difference-between-numpy-array-shape-r-1-and-r

Each element in the Tensor has the same data type, and the data type is always known. The shape (that is, the number of dimensions it has and the size of each dimension) might be only partially known. Most operations produce tensors of fully-known shapes if the shapes of their inputs are also fully known, but in some cases it’s only possible to find the shape of a tensor at graph execution time.

Some types of tensors are special, and these will be covered in other units of the TensorFlow guide. The main ones are:
*  [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable) 
*  [tf.constant](https://www.tensorflow.org/api_docs/python/tf/constant) 
*  [tf.placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) 
*  [tf.SparseTensor](https://www.tensorflow.org/api_docs/python/tf/sparse/SparseTensor) 

With the exception of [tf.Variable](https://www.tensorflow.org/api_docs/python/tf/Variable) , the value of a tensor is immutable, which means that in the context of a single execution tensors only have a single value. However, evaluating the same tensor twice can return different values; for example that tensor can be the result of reading data from disk, or generating a random number.

A Tensor object is a symbolic handle to the result of an operation, but does not actually hold the values of the operation's output. 

To find out TF profile -> [tensorflow/tensorflow/contrib/tfprof at master · tensorflow/tensorflow · GitHub](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/tfprof)

### tf.Varible


From [tensorflow doc](https://www.tensorflow.org/api_docs/python/tf/Variable)

A variable maintains state in the graph across calls to run(). You add a variable to the graph by constructing an instance of the class Variable.

The Variable() constructor requires an initial value for the variable, which can be a Tensor of any type and shape. The initial value defines the type and shape of the variable. After construction, the type and shape of the variable are fixed. The value can be changed using one of the assign methods.

If you want to change the shape of a variable later you have to use an assign Op with validate_shape=False.

In [0]:
import tensorflow as tf

In [36]:
x = tf.Variable([1.0, 2.0])
print(x)  # => <tf.Variable 'Variable_1:0' shape=(2,) dtype=float32_ref>, not value!!

<tf.Variable 'Variable_7:0' shape=(2,) dtype=float32_ref>


We need to run a session to get tensor's value, see Session, Run and Eval section below. 

More ...   
[Variable](https://databricks.com/tensorflow/variables)  
[In TensorFlow, what is the difference between Session.run() and Tensor.eval()?](https://stackoverflow.com/questions/33610685/in-tensorflow-what-is-the-difference-between-session-run-and-tensor-eval/33610914#33610914)  
[Current value of a tensor variable](https://stackoverflow.com/questions/33679382/how-do-i-get-the-current-value-of-a-variable)  
[Using tf.Print() in TensorFlow](https://towardsdatascience.com/using-tf-print-in-tensorflow-aa26e1cff11e)


### tf.constant

In [0]:
import tensorflow as tf

In [38]:
# Constant 1-D Tensor populated with value list.
aTensor = tf.constant([1, 2, 3, 4, 5, 6, 7])
print(aTensor)
print(aTensor.eval)  # this will not print out current value of the tensor, need session.

Tensor("Const_9:0", shape=(7,), dtype=int32)
<bound method Tensor.eval of <tf.Tensor 'Const_9:0' shape=(7,) dtype=int32>>


In [39]:
aTensor = tf.constant(0., shape=[2,3,4])
print(aTensor) # => Tensor("Const:0", shape=(2, 3, 4), dtype=float32), name of the variable is Const and value is 0

Tensor("Const_10:0", shape=(2, 3, 4), dtype=float32)


### tf.placeholder

The value of a placeholder tensor will always be fed by using the feed_dict optional argument to Session.run(), Tensor.eval(), or Operation.run().



#### Dictionary
A dictionary is a collection which is unordered, changeable and indexed. In Python dictionaries are written with curly brackets, and they have keys and values.

More ... [Doc](https://docs.python.org/3/tutorial/datastructures.html#dictionaries)  


In [13]:
# dictionary

# create with {} and k:v pairs
tel = {'jack': 4098, 'sape': 4139}
# tel = dict([('jack', 4098), ('sape', 4139)])   # this is another way

print(tel)

# add one pair of key-value
tel['guido'] = 4127

# get all
print(tel)

# get one of them, it likes an index with [], instead of 0, 1, 2,... use key in []
print(tel['jack'])

# loop thru for getting all their value
for k, v in tel.items():
  print(k, v)

{'jack': 4098, 'sape': 4139}
{'jack': 4098, 'sape': 4139, 'guido': 4127}
4098
jack 4098
sape 4139
guido 4127


In [2]:
# random numbers

import numpy as np

rand_array_a = np.random.rand(5, 5)
print(rand_array_a)

[[0.71986121 0.01991707 0.23132077 0.32643309 0.59165003]
 [0.48918368 0.6713024  0.26964629 0.39926319 0.43950775]
 [0.99559975 0.93955715 0.1214705  0.91854885 0.8478848 ]
 [0.0584083  0.25229037 0.31746804 0.41568321 0.5622505 ]
 [0.7594273  0.45491556 0.70059004 0.70512854 0.33922967]]


In [6]:
import tensorflow as tf

x = tf.placeholder(tf.float32, shape=(5, 5))
y = tf.matmul(x, x)

with tf.Session() as sess:
  #print(sess.run(y)) # => ERROR: will fail because x was not fed.

  rand_array = np.random.rand(5, 5)
  print(sess.run(y, feed_dict={x: rand_array}))  # ok.

[[0.60779345 1.215648   1.1446795  1.9015311  1.4552698 ]
 [0.4963767  0.97697276 0.9388003  1.6166095  1.2258639 ]
 [1.1405954  1.6927263  1.48407    1.8547864  1.5270574 ]
 [0.74115604 1.4202043  1.3705989  1.8114765  1.4954237 ]
 [0.5286491  0.95301634 0.84723926 1.6967978  1.2143029 ]]


### tf.SparseTensor 疏散

TensorFlow represents a sparse tensor as three separate dense tensors: `indices`, `values`, and `dense_shape`.

More ... [Sparse Tensors and TFRecords](https://planspace.org/20170427-sparse_tensors_and_tfrecords/)

In [5]:
import tensorflow as tf

sparse = tf.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
print(sparse)   # => no value, need session or 

SparseTensor(indices=Tensor("SparseTensor/indices:0", shape=(2, 2), dtype=int64), values=Tensor("SparseTensor/values:0", shape=(2,), dtype=int32), dense_shape=Tensor("SparseTensor/dense_shape:0", shape=(2,), dtype=int64))


## TensorFlow

### Session, Run and Eval

In [40]:
x = tf.Variable([1.0, 2.0])

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    v = sess.run(x)
    print(v)  # it will show you the value of variable. => [1. 2.]

[1. 2.]


In [58]:
t = tf.constant(2.0)
u = tf.constant(3.0)
tu = tf.multiply(t, u)
ut = tf.multiply(u, t)

init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  print(tu, ut)       # no value, they are only a graph
  print('tu', tu.eval())  # runs one step
  print('ut', ut.eval())  # runs one step
  sess.run([tu, ut])  # evaluates both tensors in a single step
  print(sess.run([tu, ut]))

Tensor("Mul_28:0", shape=(), dtype=float32) Tensor("Mul_29:0", shape=(), dtype=float32)
tu 6.0
ut 6.0
[6.0, 6.0]


### InteractiveSession
An InteractiveSession installs itself as the default session on construction. The methods tf.Tensor.eval and tf.Operation.run will use that session to run ops.

This is convenient in interactive shells and IPython notebooks, as it avoids having to pass an explicit Session object to run ops.

[a shortcut for session in Jupyter](https://www.tensorflow.org/api_docs/python/tf/InteractiveSession)

In [8]:
sess = tf.InteractiveSession()
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
# We can just use 'c.eval()' without passing 'sess'
print(c.eval())
sess.close()

30.0


In [7]:
sess = tf.InteractiveSession()

sparse = tf.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
dense = tf.sparse_tensor_to_dense(sparse)

print(dense.eval())   # ok 
# print(sess.run(dense))   # this is ok too 

sess.close()

[[1 0 0 0]
 [0 0 2 0]
 [0 0 0 0]]
